In [1]:
from grammar_ru.common import Separator
import pandas as pd

df = Separator.separate_string('Она была она красива. Он любил красивые вещи. Вещи, нитрокраситель и нитроэмаль!')
df['check_requested'] = True
df

,word_id,sentence_id,word_index,word_offset,word,word_type,word_length,check_requested
0,0,0,0,0,Она,ru,3,True
1,1,0,1,4,была,ru,4,True
2,2,0,2,9,она,ru,3,True
3,3,0,3,13,красива,ru,7,True
4,4,0,4,20,.,unk,1,True
5,5,1,0,22,Он,ru,2,True
6,6,1,1,25,любил,ru,5,True
7,7,1,2,31,красивые,ru,8,True
8,8,1,3,40,вещи,ru,4,True
9,9,1,4,44,.,unk,1,True


In [2]:
import grammar_ru.algorithms.repetitions as rp
import importlib
importlib.reload(rp)
pass

In [3]:
pd.options.display.max_rows=100

In [4]:
self=rp.RepetitionsAlgorithm(20)

In [5]:
import pandas as pd

def generate_merge_index(self, df):
    ldf = df.loc[df.check_requested]
    merge_index = []
    for i in df.word_id:
        for j in range(max(0, i - self.vicinity), i):
            merge_index.append((i, j))
    mdf = pd.DataFrame(merge_index, columns=['word_id', 'another_id'])
    norm_id = list(df.loc[df.word_type=='ru'].word_id)
    mdf = mdf.loc[mdf.word_id.isin(norm_id)]
    mdf = mdf.loc[mdf.another_id.isin(norm_id)]
    return mdf
mdf = generate_merge_index(self, df)

In [6]:
def get_word_id_to_word(self, df):
    return df.set_index('word_id').word.str.lower().to_frame('value')

word_df = get_word_id_to_word(self, df)

In [7]:
def compare(self, merge_df, word_df):
    cdf = merge_df.merge(word_df, left_on = 'word_id', right_index=True)
    cdf = cdf.merge(word_df, left_on = 'another_id', right_index=True)
    cdf['match'] = cdf.value_x == cdf.value_y
    errors = cdf.groupby('word_id').match.any()
    return list(errors.loc[errors].index)

compare(self, mdf, word_df)

[2, 10]

In [8]:
from grammar_ru.externals import PyMorphyFeaturizer

pym = PyMorphyFeaturizer()

nform_df = pym.create_features(df).set_index('word_id').normal_form.to_frame('value')
nform_df

,value
word_id,
0,она
1,быть
2,она
3,красивый
4,.
5,он
6,любить
7,красивый
8,вещь


In [9]:
compare(self, mdf, nform_df)

[2, 7, 10]

In [10]:
from grammar_ru.externals import TikhonovDict

tic = TikhonovDict.read_as_df()

In [11]:
tic = tic.loc[tic.morpheme_type=='ROOT']

In [12]:
tic = tic.set_index('word').morpheme.to_frame('value')

In [18]:
#tic.loc[tic.value=='нитр']

In [14]:
nform_df

,value
word_id,
0,она
1,быть
2,она
3,красивый
4,.
5,он
6,любить
7,красивый
8,вещь


In [15]:
tdf = nform_df.rename(columns={'value':'word'}).merge(tic, left_on='word', right_index=True).drop('word',axis=1)
tdf

,value
word_id,
0,он
2,он
1,бы
3,крас
7,крас
5,он
6,люб
8,вещь
10,вещь


In [20]:
compare(self, mdf, tdf)

[2, 5, 7, 10, 12, 14]